### **NYC Traffic Prediction Data Processing**

#### **1. Loading the Data**
First, we load the dataset containing automated traffic volume counts for NYC into a Pandas DataFrame and inspect its structure.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('ny_data.csv')

# Display dataset info and first few rows
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712605 entries, 0 to 1712604
Data columns (total 14 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   RequestID  int64 
 1   Boro       object
 2   Yr         int64 
 3   M          int64 
 4   D          int64 
 5   HH         int64 
 6   MM         int64 
 7   Vol        int64 
 8   SegmentID  int64 
 9   WktGeom    object
 10  street     object
 11  fromSt     object
 12  toSt       object
 13  Direction  object
dtypes: int64(8), object(6)
memory usage: 182.9+ MB


,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction
0,32970,Queens,2021,4,30,2,0,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
1,32970,Queens,2021,4,30,2,15,1,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
2,32970,Queens,2021,4,30,2,30,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
3,32970,Queens,2021,4,30,2,45,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
4,32970,Queens,2021,4,30,3,0,1,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB


#### **2. Handling Negative Values**
Traffic volume counts should not be negative. We check for negative values in the 'Vol' column and replace them with zeros.

In [2]:
# Count negative values
negative_count = (df['Vol'] < 0).sum()
negative_count

1

In [3]:
# Replace negative values with 0
df.loc[df['Vol'] < 0, 'Vol'] = 0

#### **3. Analyzing Missing Data**
We calculate the number of missing values in each column and the percentage of missing data. We also display rows with missing values.

In [4]:
# Count missing values
missing_counts = df.isnull().sum()

# Create a summary of missing values
missing_summary = df.isnull().sum().to_frame('missing_count')
missing_summary['missing_percentage'] = (df.isnull().sum() / len(df) * 100).round(2)

# Sort by most missing values
missing_summary = missing_summary.sort_values('missing_count', ascending=False)

print(missing_summary)


           missing_count  missing_percentage
toSt                1246                0.07
RequestID              0                0.00
Boro                   0                0.00
Yr                     0                0.00
M                      0                0.00
D                      0                0.00
HH                     0                0.00
MM                     0                0.00
Vol                    0                0.00
SegmentID              0                0.00
WktGeom                0                0.00
street                 0                0.00
fromSt                 0                0.00
Direction              0                0.00


#### **4. Handling Missing `toSt` Values**
The `toSt` (destination street) column has missing values. We identify rows where the `fromSt` (origin street) column contains the phrase 'Between ... and ...' and infer the `toSt` values from the `fromSt` column.

In [5]:
# Find rows where 'fromSt' contains 'Between ... and ...'
df['IsTargetRow'] = df['fromSt'].str.contains(r'^Between .+ and .+$', regex=True, na=False) & df['toSt'].isna()

# Display the rows where the condition is True
print(df[df['IsTargetRow']])

# Extract 'toSt' from 'fromSt'
matching_rows = df[df['IsTargetRow']].copy()
matching_rows['toSt'] = matching_rows['fromSt'].str.extract(r'and (.+)$')[0]

# Update the original DataFrame
df.loc[df['IsTargetRow'], 'toSt'] = matching_rows['toSt']

# Drop the helper column
df.drop(columns=['IsTargetRow'], inplace=True)

# Display the updated DataFrame
print(df)

         RequestID      Boro    Yr  M   D  HH  MM  Vol  SegmentID   
426467       10877  Brooklyn  2009  5   9  23  15  266     230717  \
426468       10877  Brooklyn  2009  5   9  23  30  230     230717   
426469       10877  Brooklyn  2009  5   9  23  45  194     230717   
426470       10877  Brooklyn  2009  5  10   0   0  196     230717   
426471       10877  Brooklyn  2009  5  10   0  15  193     230717   
...            ...       ...   ... ..  ..  ..  ..  ...        ...   
1662446      10877  Brooklyn  2009  5  19  10  45  270     230717   
1662447      10877  Brooklyn  2009  5  19  11   0  230     230717   
1662448      10877  Brooklyn  2009  5  19  11  15  294     230717   
1662449      10877  Brooklyn  2009  5  19  11  30  275     230717   
1662450      10877  Brooklyn  2009  5  19  11  45  302     230717   

                           WktGeom                  street   
426467   POINT (993612.8 158332.9)  Ocean Pkwy SB Mainline  \
426468   POINT (993612.8 158332.9)  Ocean Pkwy 

#### **5. Further Cleaning of `fromSt` and `toSt`**
We clean the `fromSt` column to extract only the origin street name and update the `toSt` column accordingly.

In [6]:
# Clean 'fromSt' by extracting only the origin street
df['fromSt'] = df['fromSt'].str.replace(r'^Between (.+) and .+$', r'\1', regex=True)

# Update 'toSt' values where necessary
condition = df['fromSt'].str.contains(r' and ', na=False) & df['toSt'].isna()
df.loc[condition, 'toSt'] = df.loc[condition, 'fromSt'].str.extract(r'and (.+)$')[0]
df.loc[condition, 'fromSt'] = df.loc[condition, 'fromSt'].str.extract(r'^(.+) and')[0]

# Display the cleaned DataFrame
print(df)


         RequestID    Boro    Yr  M   D  HH  MM  Vol  SegmentID   
0            32970  Queens  2021  4  30   2   0    0     149701  \
1            32970  Queens  2021  4  30   2  15    1     149701   
2            32970  Queens  2021  4  30   2  30    0     149701   
3            32970  Queens  2021  4  30   2  45    0     149701   
4            32970  Queens  2021  4  30   3   0    1     149701   
...            ...     ...   ... ..  ..  ..  ..  ...        ...   
1712600      17447  Queens  2014  6  28  15   0    6      90610   
1712601      17447  Queens  2014  6  28  15  15    7      90610   
1712602      17447  Queens  2014  6  28  15  30    8      90610   
1712603      17447  Queens  2014  6  28  15  45    8      90610   
1712604      17447  Queens  2014  6  28  16   0    6      90610   

                                              WktGeom          street   
0        POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE  \
1        POINT (997407.0998491726 208620.92612708

#### Checking for missing values again

In [7]:
missing_counts = df.isnull().sum()
missing_counts

RequestID    0
Boro         0
Yr           0
M            0
D            0
HH           0
MM           0
Vol          0
SegmentID    0
WktGeom      0
street       0
fromSt       0
toSt         0
Direction    0
dtype: int64

### **Summary**
In this script, we:
1. Loaded the traffic volume dataset.
2. Handled negative values in the traffic volume column.
3. Analyzed and cleaned missing values, particularly for the `toSt` column.
4. Applied regex to clean and infer missing `toSt` values from `fromSt`.
5. Further cleaned the `fromSt` column and finalized the dataset.